5.0 - Chuẩn bị dữ liệu

In [2]:
import pandas as pd
df_full = pd.read_csv('../data_clean/full_merged_data.csv')
df_thong_tin = pd.read_csv('../data_clean/sensor_info_clean.csv')
df_bao_tri = pd.read_csv('../data_clean/maintenance_log_clean.csv')

5.1 - Pivot giá trị đo trung bình theo sensor_type × location

In [3]:
pivot_gia_tri = df_full.pivot_table(values='value', index='sensor_type', columns='location', aggfunc='mean')
pivot_gia_tri

location,Line1
sensor_type,
Current,38.571429


5.2 - Pivot tổng downtime_hours theo machine_id × issue_type

In [4]:
pivot_downtime = df_full.pivot_table(values='downtime_hours', index='machine_id', columns='issue_type', aggfunc='sum')
pivot_downtime

issue_type,Electrical,Overheat,Vibration
machine_id,,,
MC01,80.0,48.0,80.0
